In [1]:
#import dependencies
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
!pip install pandas numpy

In [2]:
#This is the file I am reading in for my analysis. 
mental_health_csv = "./Resources/Conference and map Aishath Shahama Full PhD Study June 2022 to Dec 2023.csv"
country_csv = "./Resources/country_long_lat_data.csv"

In [3]:
#Pulling in the data here
mental_health_df = pd.read_csv(mental_health_csv)
country_df = pd.read_csv(country_csv)

In [4]:
mental_health_df.head()

,Religious Affliation,South Asian Heritage,Cause of Mental Health problems,Treatability,Treatments,Experience or Wittnessed stigma,Incident Description
0,Islam,Maldives,Lifestyle and genetics.,No,They can be brought under control but not cured.,Experienced,Was told to go off antidepressants because dep...
1,NaN,Maldives,Genetics and mentality,No,NaN,Experienced,Annoying in time
2,Hindhu,India,Genetics,No,It's something you are born with,Experienced,NaN
3,Islam,Maldives,"Genetics, environment and trauma",Yes,"With medication, therapy and a lot of will power",Experienced,Yes
4,Islam,Maldives,Stress and uncertainty,Yes,Psychoherapy and psychopharmaceuticals,Experienced,When saying I am going to consult a Psychologi...


In [5]:
mental_health_df.columns

Index(['Religious Affliation', 'South Asian Heritage',
       'Cause of Mental Health problems', 'Treatability', 'Treatments',
       'Experience or Wittnessed stigma', 'Incident Description'],
      dtype='object')

In [11]:
mental_health_df.count()

Religious Affliation               509
South Asian Heritage               523
Cause of Mental Health problems    494
Treatability                       523
Treatments                         407
Experience or Wittnessed stigma    521
Incident Description               192
dtype: int64

In [21]:
#What were the responses to this question? 
mental_health_df['South Asian Heritage'].unique()

array(['Maldives', 'India', 'Pakistan', 'Nepal', 'Bangladesh',
       'Afghanistan', 'Sri Lanka'], dtype=object)

In [15]:
# Boundaries for some countries
country_bounds = {
    "Pakistan": {"lat_min": 23.6345, "lat_max": 37.0841, "lon_min": 60.8728, "lon_max": 77.6572},
    "India": {"lat_min": 6.4626, "lat_max": 37.284, "lon_min": 68.1166, "lon_max": 97.4026},
    "Maldives": {"lat_min": -0.749, "lat_max": 7.100, "lon_min": 72.508, "lon_max": 73.707},
    "Bangladesh": {"lat_min": 20.250, "lat_max": 26.638, "lon_min": 88.041, "lon_max": 92.675},
    "Nepal": {"lat_min": 26.347, "lat_max": 30.447, "lon_min": 80.058, "lon_max": 88.201},
    "Sri Lanka": {"lat_min": 5.875, "lat_max": 9.83, "lon_min": 79.958, "lon_max": 81.845},
    "Afghanistan": {"lat_min": 29.30, "lat_max": 38.30, "lon_min": 61.31, "lon_max": 75.00}
}



In [17]:
import random

def get_random_coords(country):
    bounds = country_bounds.get(country)
    if bounds:
        lat = random.uniform(bounds['lat_min'], bounds['lat_max'])
        lon = random.uniform(bounds['lon_min'], bounds['lon_max'])
        return lat, lon
    return None, None

In [19]:
# Function to apply random coordinate generation
def assign_random_coordinates(row):
    country = row['South Asian Heritage'] 
    lat, lon = get_random_coords(country)
    return pd.Series([lat, lon])

In [23]:
# Create new columns in the DataFrame
mental_health_df[['latitude', 'longitude']] = mental_health_df.apply(assign_random_coordinates, axis=1)

In [25]:
mental_health_df.head()

,Religious Affliation,South Asian Heritage,Cause of Mental Health problems,Treatability,Treatments,Experience or Wittnessed stigma,Incident Description,latitude,longitude
0,Islam,Maldives,Lifestyle and genetics.,No,They can be brought under control but not cured.,Experienced,Was told to go off antidepressants because dep...,4.932196,72.562659
1,NaN,Maldives,Genetics and mentality,No,NaN,Experienced,Annoying in time,-0.308401,73.116975
2,Hindhu,India,Genetics,No,It's something you are born with,Experienced,NaN,29.257034,68.457540
3,Islam,Maldives,"Genetics, environment and trauma",Yes,"With medication, therapy and a lot of will power",Experienced,Yes,-0.462930,73.481464
4,Islam,Maldives,Stress and uncertainty,Yes,Psychoherapy and psychopharmaceuticals,Experienced,When saying I am going to consult a Psychologi...,6.891998,73.372696


In [27]:
# Save the updated DataFrame to a new CSV file
mental_health_df.to_csv('SouthAsianMentalHealth_with_country_data.csv', index=False)

In [29]:
# Optional: Display the updated DataFrame
print(mental_health_df.head())

  Religious Affliation South Asian Heritage   Cause of Mental Health problems  \
0                Islam             Maldives           Lifestyle and genetics.   
1                  NaN             Maldives            Genetics and mentality   
2               Hindhu                India                          Genetics   
3                Islam             Maldives  Genetics, environment and trauma   
4                Islam             Maldives            Stress and uncertainty   

  Treatability                                        Treatments  \
0           No  They can be brought under control but not cured.   
1           No                                               NaN   
2           No                  It's something you are born with   
3          Yes  With medication, therapy and a lot of will power   
4          Yes            Psychoherapy and psychopharmaceuticals   

  Experience or Wittnessed stigma  \
0                     Experienced   
1                     Experien